# 🌾 Image2Biomass Prediction

## Project Overview

This notebook implements a **6-model ensemble learning system** which is used to predict dry biomass  by combining  imagery and environmental tabular features, which is  **Cheap, affordable and better** then other large models.

### Architecture

- **Vision Models** (3): ViT-Base, ResNet-50, DenseNet-121 for image patch processing  
- **Species Classification Model**: ViT-Base (16-class classifier) for predicting pasture species from imagery
- **Tabular Model**: XGBoost trained on environmental and spectral features  
- **Meta-Learner**: Fully connected neural network that fuses base model predictions and tabular features  

```
                            System Parameter Breakdown:
                            ─────────────────────────────
                            1. ViT-Base (Biomass):      86.6M
                            2. ResNet-50 (Biomass):     25.6M
                            3. DenseNet-121 (Biomass):  8.0M
                            4. Species Classifier:      86.6M
                            5. Meta-Learner:            5.1K
                            ─────────────────────────────
                            Total Parameters:          ~207M

```

### Data Processing

- **Input images**: 2000×1000 pixel  images  
- **Patch extraction**: Splits each image into 8 patches (500×500 pixels), resized to 224×224 for model inputs  
- **Tabular features**: 29-dimensional vector combining:
  - Numeric features: NDVI, height, month, day of year, quarter (normalized)
  - Categorical features: state (4 dims), species (15 dims), target type (5 dims)
- **Targets**: Predicts 5 biomass target types — Dry_Green_g, Dry_Dead_g, Dry_Clover_g, Dry_Total_g, GDM_g  

### Species Classification Model

The **Species ViT** is a ViT-Base model trained for 15-class species classification:

- **Input**: Same 8 image patches as biomass models (224×224 each)
- **Output**: Species class prediction (15 unique species)
- **Purpose**: Predicts pasture species from satellite imagery for intelligent feature fallback
- **Usage**: During test inference with missing features, species prediction improves fallback feature generation accuracy (Level 1 , 2 & 3 fallback levels)
- **Training**: Trained on training set species labels, achieves high accuracy to ensure reliable predictions

### Key Features

✅ **6-model ensemble** (3 vision + 1 species + 1 tabular + 1 meta-learner)  
✅ **Intelligent multi-level fallback** for missing test features using species prediction and statistical inference  
✅ **Species-informed feature generation** — Uses predicted species to improve tabular feature accuracy in test data  
✅ **Patch-level image processing** to capture spatial detail and improve robustness  
✅ **GPU memory optimization and model offloading** for efficient training  
✅ **Comprehensive logging** for detailed training and evaluation tracking  
✅ **Weighted R² loss** optimized per competition evaluation metrics (0.5 for Dry_Total_g, 0.2 for GDM, 0.1 for others)  
✅ **Cross-validation on 85/15 train/val split** to monitor generalization  
✅ **Robust ensemble averaging and meta-learning** for final biomass prediction  

### Data Pipeline

```
    
                                            Test Image
                                                ↓
                            Load Image → Crop 8 Patches (500×500 → 224×224)
                                                ↓
                                ┌─────────────────────────────────┐
                                │ Species Model Prediction        │
                                │ (15 classes: pasture species)   │
                                └────────────┬────────────────────┘
                                             ↓ (species label)
                                ┌─────────────────────────────────┐
                                │ Intelligent Feature Fallback    │
                                │ Level 1: Random state + species │
                                │ Level 2: Exact sample ID match  │
                                │ Level 3: Image ID + species     │
                                │ Level 4: Statistical generation │
                                └────────────┬────────────────────┘
                                             ↓ (tabular features)
                            ┌─────────────────────────────────────────┐
                            │ Base Models                             │
                            │ - Vision: ViT, ResNet, DenseNet (X pred)│
                            │ - Tabular: XGBoost (1 pred)             │
                            │ Outputs: 1 predictions per model        │
                            └────────────┬────────────────────────────┘
                                         ↓ (X base predictions)
                            ┌─────────────────────────────────────────┐
                            │ Meta-Learner                            │
                            │ Input: X predictions + 29 features      │
                            │ Output: 1 final prediction              │
                            └────────────────┬────────────────────────┘
                                             ↓
                                Final Biomass Prediction (grams)

```

### Expected Performance

- Optimizes globally weighted R² metric reflecting competition's scoring  
- Species model improves test feature fallback accuracy by providing accurate species labels
- Ensemble diversity (vision + tabular + species) provides robustness  
- Meta-learner learns optimal combination of base model predictions  
- Comprehensive validation monitoring with early stopping prevents overfitting  

### Output Format

- **File**: CSV with `sample_id` and `target` columns
- **Rows**: 5 rows (one per target type: Dry_Green_g, Dry_Dead_g, Dry_Clover_g, GDM_g, Dry_Total_g)
- **Values**: Biomass predictions in grams (float, non-negative, rounded to 3 decimal places)
- **Evaluation**: Weighted R² computed globally across all rows combined

---

**Status**: ✅ Ready | **Models**: 6 | **Features**: 29 | **Targets**: 5

##  1: IMPORTS & CONFIGURATION

This cell initializes all required dependencies for the complete pipeline.

### Libraries

#### Data Processing & ML
- `pandas`, `numpy`: Data manipulation
- `scikit-learn`: Preprocessing, metrics, splitting
- `xgboost`: Tree-based ensemble model

#### Deep Learning
- `torch`, `torchvision`: PyTorch framework
- `timm`: Vision model library (ViT, ResNet, DenseNet)
- `torch.nn.functional`: Neural network operations

#### Visualization & Logging
- `matplotlib`, `seaborn`: Data visualization
- `PIL`: Image processing
- `logging`, `tqdm`: Progress tracking
- `cv2`: Computer vision operations

#### Utilities
- `pathlib`: File path management
- `json`, `pickle`: Serialization
- `shutil`, `gc`: System utilities
- `datetime`: Timestamps
- `socket`: Network connectivity checks

### Environment Detection

- GPU availability check
- Device assignment (CUDA/CPU)
- Random seed initialization for reproducibility


In [ ]:
import os
import gc
import re
import sys
import json
import math
import socket
import pickle
import joblib
import shutil
import random 
import zipfile
import logging
import warnings
import traceback
from pathlib import Path
warnings.filterwarnings('ignore')
os.environ['OPENCV_LOG_LEVEL'] = 'OFF'

# Image processing
import cv2
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model
import torch.nn.functional as FF
from torchvision import transforms
import torchvision.transforms.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50, densenet121

# Progress tracking
import time
from tqdm import tqdm
from collections import Counter
from datetime import datetime, timedelta

# Data processing
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy import stats
from scipy.stats import trim_mean
from scipy.stats import randint, uniform
from sklearn.preprocessing import RobustScaler
from scipy.ndimage import laplace, gaussian_filter
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

for h in logging.root.handlers[:]:
    logging.root.removeHandler(h)
logger = logging.getLogger(__name__)
for h in logger.handlers[:]:
    logger.removeHandler(h)

logger.propagate = False
logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler(stream=sys.stdout)
    handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

log_file_path = "/kaggle/working/pipeline.log" 
file_handler = logging.FileHandler(log_file_path, mode="w", encoding="utf-8")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logger.addHandler(file_handler)

logger.info("=" * 80)
logger.info("PASTURE BIOMASS PREDICTION - 6 MODEL ENSEMBLE PIPELINE")
logger.info("=" * 80)
start_time = datetime.now()
logger.info(f"Execution started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
logger.info("=" * 80)